In [10]:
import sys        
import random
import warnings
import time
import numpy as np
import pandas as pd
import collections, functools, operator
import os
import pickle

# Parameters of system
warnings.filterwarnings("ignore")

In [12]:
os.chdir('/Users/moises/Desktop/Produccion-Tesis/')

replicas = 15

for experiment in [1,2,3,4,5,6]:            
    file_name = f"Resultados/Escenario {experiment}/"
    df = pd.read_excel(file_name+"objective_value.xlsx")
    print(f"Escenario {experiment}: {df['value'].sum()/replicas} +- {df['value'].std()*1.96/np.sqrt(replicas)}")

Escenario 1: 92077730.73913124 +- 47049.654855528956
Escenario 2: 92016729.55591758 +- 48169.579435299725
Escenario 3: 91973541.09359795 +- 48424.408873779554
Escenario 4: 92030164.2246732 +- 49179.85530493181
Escenario 5: 92129810.14701323 +- 47957.90494666894
Escenario 6: 92160517.86538306 +- 46801.14129993734


In [ ]:
file_name = f"Resultados/Escenario 1/"
df1 = pd.read_excel(file_name+"objective_value.xlsx")

file_name = f"Resultados/Escenario 2/"
df2 = pd.read_excel(file_name+"objective_value.xlsx")

df = df1.merge(df2, on=['t','r'])
df['dif'] = df.value_x - df.value_y
print(f"{df.groupby('r').sum()['dif'].mean()} + {df.groupby('r').sum()['dif'].std()*1.96/np.sqrt(10)}")

In [ ]:
final = pd.DataFrame()
for experiment in range (1, 7):
    # Recorrer cada experimento
    if experiment in [1, 2,  3, 4, 5, 6]:
        file_name = f"Resultados/Escenario {experiment}/"
        df = pd.read_excel(file_name+"objective_value.xlsx").rename(columns={'value': 'utilidad'}).groupby(['r']).sum().reset_index()['utilidad']
        df1 = pd.read_excel(file_name+"ingresos.xlsx").rename(columns={'value': 'ingresos'}).groupby(['r']).sum().reset_index()['ingresos']
        df2 = pd.read_excel(file_name+"costo_inventario.xlsx").rename(columns={'value': 'costo_inventario'}).groupby(['r']).sum().reset_index()['costo_inventario']
        df3 = pd.read_excel(file_name+"costo_corte.xlsx").rename(columns={'value': 'costo_corte'}).groupby(['r']).sum().reset_index()['costo_corte']
        df4 = pd.read_excel(file_name+"costo_merma.xlsx").rename(columns={'value': 'costo_merma'}).groupby(['r']).sum().reset_index()['costo_merma']
        df = pd.concat([df, df1, df2, df3, df4], axis=1)
        df['Escenario'] = f"Escenario {experiment}"
        df = df.reset_index().rename(columns={'index': 'r'})
        df['r'] += 1

        # Se agrega al DataFrame 
        final = pd.concat([final, df])
        
        fila = f"Escenario {experiment} & "
        for metrica in ['utilidad', 'ingresos', 'costo_inventario', 'costo_corte', 'costo_merma']:
            # Calcula la media y desviación estándar de la columna 'utilidad'
            media = df[metrica].mean()
            desviacion_estandar = df[metrica].std()

            # Calcula el tamaño de la muestra
            tamano_muestra = len(df)

            # Define el nivel de confianza deseado (por ejemplo, 95%)
            nivel_confianza = 0.95

            # Calcula el valor crítico de la distribución t de Student
            valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, tamano_muestra - 1)

            # Calcula el error estándar
            error_estandar = desviacion_estandar / np.sqrt(tamano_muestra)

            # Calcula el intervalo de confianza
            intervalo_inferior = media - valor_critico * error_estandar
            intervalo_superior = media + valor_critico * error_estandar

            # Expresa el intervalo de confianza en potencia de 10e6 (potencia de 6)
            media = "{:.4f}".format(media * 1e-6)
            error = "{:.4f}".format(valor_critico * error_estandar * 1e-6)

            # Imprime el intervalo de confianza expresado en potencia de 10e6 (potencia de 6)
            cadena = f"${media}$ $\pm$ ${error}$ $\\times10^-6_$".replace('-', '{'). replace('_', '}')
            #print(f"{metrica}:{cadena}")

            fila += f"{cadena} & "

        fila =  fila[:-3]
        if experiment != 6:
            fila += "\\\\ \\midrule "
        else:
            fila += "\\\\ \\bottomrule "

        print(fila)

        
#final
final.to_excel("/Users/moises/Desktop/Codigo - Test significancia/ANOVA.xlsx", sheet_name="ANOVA")    
